<a href="https://cocl.us/Data_Science_with_Scalla_top"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/SC0103EN/adds/Data_Science_with_Scalla_notebook_top.png" width = 750, align = "center"></a>
 <br/>
<a><img src="https://ibm.box.com/shared/static/ugcqz6ohbvff804xp84y4kqnvvk3bq1g.png" width="200" align="center"></a>"

# Module 5: Pipeline and Grid Search

## Predicting Grant Applications: Building a Pipeline

### Lesson Objectives

* After completing this lesson, you should be able to extract useful information from the results of the grid search, including:
  - the average area under the ROC curve for each combination of parameters
  - the parameters of the best model
  - the feature importances of the best model
  
### avgMetrics

In [1]:
import org.apache.spark.sql.SparkSession
val spark = SparkSession.builder().getOrCreate()
import spark.implicits._
import org.apache.spark.sql.functions._

val data = spark.read.
  format("com.databricks.spark.csv").
  option("delimiter", "\t").
  option("header", "true").
  option("inferSchema", "true").
//   load("/resources/data/grantsPeople.csv")
  load("grantsPeople.csv")

data.show()

Intitializing Scala interpreter ...

Spark Web UI available at http://host.docker.internal:4044
SparkContext available as 'sc' (version = 3.3.0, master = local[*], app id = local-1670056426962)
SparkSession available as 'spark'


+--------------------+----------+---------------+---------+--------------+---------+--------------------+-------------+----------------+-------------+--------+----------+--------+-----------------------------------+--------------------------+----------------------------+----+----+----+----+------------+------------+--------------------+-------------------+
|Grant_Application_ID| RFCD_Code|RFCD_Percentage| SEO_Code|SEO_Percentage|Person_ID|                Role|Year_of_Birth|Country_of_Birth|Home_Language| Dept_No|Faculty_No|With_PHD|No_of_Years_in_Uni_at_Time_of_Grant|Number_of_Successful_Grant|Number_of_Unsuccessful_Grant|  A2|   A|   B|   C|Grant_Status|Sponsor_Code| Contract_Value_Band|Grant_Category_Code|
+--------------------+----------+---------------+---------+--------------+---------+--------------------+-------------+----------------+-------------+--------+----------+--------+-----------------------------------+--------------------------+----------------------------+----+----+-

import org.apache.spark.sql.SparkSession
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@72594949
import spark.implicits._
import org.apache.spark.sql.functions._
data: org.apache.spark.sql.DataFrame = [Grant_Application_ID: int, RFCD_Code: string ... 22 more fields]


In [2]:

val researchers = data.
  withColumn ("phd", data("With_PHD").equalTo("Yes").cast("Int")).
  withColumn ("CI", data("Role").equalTo("CHIEF_INVESTIGATOR").cast("Int")).
  withColumn("paperscore", data("A2") * 4 + data("A") * 3)

val grants = researchers.groupBy("Grant_Application_ID").agg(
  max("Grant_Status").as("Grant_Status"),
  max("Grant_Category_Code").as("Category_Code"),
  max("Contract_Value_Band").as("Value_Band"),
  sum("phd").as("PHDs"),
  when(max(expr("paperscore * CI")).isNull, 0).
    otherwise(max(expr("paperscore * CI"))).as("paperscore"),
  count("*").as("teamsize"),
  when(sum("Number_of_Successful_Grant").isNull, 0).
    otherwise(sum("Number_of_Successful_Grant")).as("successes"),
  when(sum("Number_of_Unsuccessful_Grant").isNull, 0).
    otherwise(sum("Number_of_Unsuccessful_Grant")).as("failures")
)

grants.show()

+--------------------+------------+-------------+--------------------+----+----------+--------+---------+--------+
|Grant_Application_ID|Grant_Status|Category_Code|          Value_Band|PHDs|paperscore|teamsize|successes|failures|
+--------------------+------------+-------------+--------------------+----+----------+--------+---------+--------+
|                   1|           1|  GrantCatUnk|  ContractValueBandA|null|        22|       1|        0|       0|
|                   2|           1|  GrantCat10A|  ContractValueBandB|   1|        60|       1|        0|       0|
|                   3|           1|  GrantCat10B|  ContractValueBandA|   2|        13|       7|        0|       0|
|                   4|           1|  GrantCat10B|  ContractValueBandC|   2|         0|       6|        0|       0|
|                   5|           0|  GrantCat10A|  ContractValueBandA|   1|        12|       1|        0|       0|
|                   6|           1|  GrantCat10A|ContractValueBandUnk|   1|     

researchers: org.apache.spark.sql.DataFrame = [Grant_Application_ID: int, RFCD_Code: string ... 25 more fields]
grants: org.apache.spark.sql.DataFrame = [Grant_Application_ID: int, Grant_Status: int ... 7 more fields]


In [3]:

import org.apache.spark.ml.feature.StringIndexer

val value_band_indexer = new StringIndexer().
  setInputCol("Value_Band").
  setOutputCol("Value_index").
  fit(grants)
  
val category_indexer = new StringIndexer().
  setInputCol("Category_Code").
  setOutputCol("Category_index").
  fit(grants)
  
val label_indexer = new StringIndexer().
  setInputCol("Grant_Status").
  setOutputCol("status").
  fit(grants)



import org.apache.spark.ml.feature.StringIndexer
value_band_indexer: org.apache.spark.ml.feature.StringIndexerModel = StringIndexerModel: uid=strIdx_2ac4c5292f7f, handleInvalid=error
category_indexer: org.apache.spark.ml.feature.StringIndexerModel = StringIndexerModel: uid=strIdx_8ce2dc20f1b0, handleInvalid=error
label_indexer: org.apache.spark.ml.feature.StringIndexerModel = StringIndexerModel: uid=strIdx_a826b719fc96, handleInvalid=error


In [4]:
import org.apache.spark.ml.feature.VectorAssembler

val assembler = new VectorAssembler().
  setInputCols(Array(
    "Value_index"
    ,"Category_index"
    ,"PHDs"
    ,"paperscore"
    ,"teamsize"
    ,"successes"
    ,"failures"
  )).setOutputCol("assembled")


import org.apache.spark.ml.feature.VectorAssembler
assembler: org.apache.spark.ml.feature.VectorAssembler = VectorAssembler: uid=vecAssembler_e02f15c634ba, handleInvalid=error, numInputCols=7


In [5]:
import org.apache.spark.ml.classification.RandomForestClassifier
import org.apache.spark.ml.classification.RandomForestClassificationModel

val rf = new RandomForestClassifier().
  setFeaturesCol("assembled").
  setLabelCol("status").
  setSeed(42)

import org.apache.spark.ml.Pipeline
val pipeline = new Pipeline().setStages(Array(
    value_band_indexer,
    category_indexer,
    label_indexer,
    assembler,
    rf)
  )

import org.apache.spark.ml.classification.RandomForestClassifier
import org.apache.spark.ml.classification.RandomForestClassificationModel
rf: org.apache.spark.ml.classification.RandomForestClassifier = rfc_b4afc126417c
import org.apache.spark.ml.Pipeline
pipeline: org.apache.spark.ml.Pipeline = pipeline_24a3229c37aa


In [6]:
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
val auc_eval = new BinaryClassificationEvaluator().
  setLabelCol("status").
  setRawPredictionCol("rawPrediction")

auc_eval.getMetricName

val tr = grants.filter("Grant_Application_ID < 6635")
val te = grants.filter("Grant_Application_ID >= 6635")
val training = tr.na.fill(0, Seq("PHDs"))
val test = te.na.fill(0, Seq("PHDs"))

val model = pipeline.fit(training)
val pipeline_results = model.transform(test)
auc_eval.evaluate(pipeline_results)

rf.extractParamMap


import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
auc_eval: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = BinaryClassificationEvaluator: uid=binEval_34c90dcf5d9a, metricName=areaUnderROC, numBins=1000
tr: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Grant_Application_ID: int, Grant_Status: int ... 7 more fields]
te: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Grant_Application_ID: int, Grant_Status: int ... 7 more fields]
training: org.apache.spark.sql.DataFrame = [Grant_Application_ID: int, Grant_Status: int ... 7 more fields]
test: org.apache.spark.sql.DataFrame = [Grant_Application_ID: int, Grant_Status: int ... 7 more fields]
model: org.apache.spark.ml.PipelineModel = pipeline_24a3229c37aa
pipeline_results: org.apa...


In [7]:
import org.apache.spark.ml.tuning.ParamGridBuilder

val paramGrid = new ParamGridBuilder().
  addGrid(rf.maxDepth, Array(2, 5)).
  addGrid(rf.numTrees, Array(1, 20)).
  build()

import org.apache.spark.ml.tuning.ParamGridBuilder
paramGrid: Array[org.apache.spark.ml.param.ParamMap] =
Array({
	rfc_b4afc126417c-maxDepth: 2,
	rfc_b4afc126417c-numTrees: 1
}, {
	rfc_b4afc126417c-maxDepth: 2,
	rfc_b4afc126417c-numTrees: 20
}, {
	rfc_b4afc126417c-maxDepth: 5,
	rfc_b4afc126417c-numTrees: 1
}, {
	rfc_b4afc126417c-maxDepth: 5,
	rfc_b4afc126417c-numTrees: 20
})


In [8]:
import org.apache.spark.ml.tuning.CrossValidator

val cv = new CrossValidator().
  setEstimator(pipeline).
  setEvaluator(auc_eval).
  setEstimatorParamMaps(paramGrid).
  setNumFolds(3)

val cvModel = cv.fit(training)

val cv_results = cvModel.transform(test)
cvModel.avgMetrics

import org.apache.spark.ml.tuning.CrossValidator
cv: org.apache.spark.ml.tuning.CrossValidator = cv_132be5ad91fb
cvModel: org.apache.spark.ml.tuning.CrossValidatorModel = CrossValidatorModel: uid=cv_132be5ad91fb, bestModel=pipeline_24a3229c37aa, numFolds=3
cv_results: org.apache.spark.sql.DataFrame = [Grant_Application_ID: int, Grant_Status: int ... 14 more fields]
res3: Array[Double] = Array(0.6954426991157221, 0.8729449693948864, 0.8972594318668786, 0.922704923185537)


### Finding the Winning Parameters

In [9]:
import org.apache.spark.ml.param.ParamMap
import org.apache.spark.ml.tuning.CrossValidatorModel

implicit class BestParamMapCrossValidatorModel(cvModel: CrossValidatorModel)
{
  def bestEstimatorParamMap: ParamMap = cvModel.getEstimatorParamMaps.zip(cvModel.avgMetrics).maxBy(_._2)._1
}

import org.apache.spark.ml.param.ParamMap
import org.apache.spark.ml.tuning.CrossValidatorModel
defined class BestParamMapCrossValidatorModel


### Using bestEstimatorParamMap

In [10]:

println(cvModel.bestEstimatorParamMap)


{
	rfc_b4afc126417c-maxDepth: 5,
	rfc_b4afc126417c-numTrees: 20
}


### Best Model

In [11]:
val bestPipelineModel = cvModel.bestModel.asInstanceOf[org.apache.spark.ml.PipelineModel]
bestPipelineModel.stages

bestPipelineModel: org.apache.spark.ml.PipelineModel = pipeline_24a3229c37aa
res5: Array[org.apache.spark.ml.Transformer] = Array(StringIndexerModel: uid=strIdx_2ac4c5292f7f, handleInvalid=error, StringIndexerModel: uid=strIdx_8ce2dc20f1b0, handleInvalid=error, StringIndexerModel: uid=strIdx_a826b719fc96, handleInvalid=error, VectorAssembler: uid=vecAssembler_e02f15c634ba, handleInvalid=error, numInputCols=7, RandomForestClassificationModel: uid=rfc_b4afc126417c, numTrees=20, numClasses=2, numFeatures=7)


### Extracting the Winning Classifier

In [12]:
val bestRandomForest = bestPipelineModel.stages(4).asInstanceOf[RandomForestClassificationModel]
bestRandomForest.toDebugString

bestRandomForest: org.apache.spark.ml.classification.RandomForestClassificationModel = RandomForestClassificationModel: uid=rfc_b4afc126417c, numTrees=20, numClasses=2, numFeatures=7
res6: String =
"RandomForestClassificationModel: uid=rfc_b4afc126417c, numTrees=20, numClasses=2, numFeatures=7
  Tree 0 (weight 1.0):
    If (feature 0 in {2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0,16.0})
     If (feature 1 in {1.0,2.0,4.0,5.0,6.0,7.0,8.0,9.0,11.0,12.0,13.0})
      If (feature 4 <= 7.5)
       Predict: 0.0
      Else (feature 4 > 7.5)
       Predict: 1.0
     Else (feature 1 not in {1.0,2.0,4.0,5.0,6.0,7.0,8.0,9.0,11.0,12.0,13.0})
      If (feature 0 in {9.0,13.0,14.0})
       If (feature 4 <= 3.5)
        If (feature 0 in {13.0})
         Predict: 1.0
  ...


### totalNumNodes

In [13]:
bestRandomForest.totalNumNodes

res7: Int = 736


### Feature Importances

In [14]:
bestRandomForest.featureImportances

res8: org.apache.spark.ml.linalg.Vector = (7,[0,1,2,3,4,5,6],[0.20428274601717972,0.17210062921296593,0.01360393415994755,0.025440963724379338,0.003365188203866553,0.21815889499218416,0.3630476436894768])


### Wrapping Up

* Using the default parameters, we had an area under the ROC curve of 0.909
* After a grid search, we got that up to 0.926
* Running the grid search on a cluster was a real timesaver
* Not all of our features proved very useful; maybe you can do better!

### Module Summary

* Having completed this module about Predicting Grant Applications, you should be able to:
  - Understand how to fit together the functions available in Spark's machine learning libraries to solve real problems
  - Fit models in a fraction of the time, using a Spark cluster


### About the Authors

[Petro Verkhogliad](https://www.linkedin.com/in/vpetro) is Consulting Manager at Lightbend. He holds a Masters degree in Computer Science with specialization in Intelligent Systems. He is passionate about functional programming and applications of AI.